# Boston Housing Classification Logistic Regression

In [ ]:
from pyspark.sql.types import BooleanType
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import expr
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from socket import gethostname, gethostbyname
from helpers.helper_functions import translate_to_file_string

In [ ]:
inputFile = "hdfs:///data/Boston_Housing_Data.csv"

Spark session creation 

In [ ]:
spark = (SparkSession
       .builder 
       .master("yarn")  
       .appName("BostonHousingClass")
       .getOrCreate())

DataFrame creation using an ifered Schema 

In [ ]:
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ";") \
       .csv(inputFile) \
       .withColumn("CATBOOL", expr("CAT").cast(BooleanType()))
print(df.printSchema())

Prepare training and test data.

In [ ]:
featureCols = df.columns.copy()
featureCols.remove("MEDV")
featureCols.remove("CAT")
featureCols.remove("CATBOOL") 
print(featureCols)

assembler =  VectorAssembler(outputCol="features", inputCols=featureCols)

In [ ]:
labledPointDataSet = assembler.transform(df)
splits = labledPointDataSet.randomSplit([0.9, 0.1 ], 12345)
training = splits[0]
test = splits[1]

Logistic regression

In [ ]:
#TODO Optimize the paramters 
lr = LogisticRegression(labelCol="CAT",featuresCol="features", maxIter=100, \
                           regParam=0)

Train the model 

In [ ]:
lrModel = lr.fit(training)

Test the model

In [ ]:
predictions = lrModel.transform(test)
predictions.show()

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="CAT",rawPredictionCol="rawPrediction", metricName="areaUnderROC")
accuracy = evaluator.evaluate(predictions)
print("Test Error",(1.0 - accuracy))